In [63]:
import cifar10
import numpy as np
import tensorflow as tf

In [64]:
cifar10.data_path = "data/CIFAR-10/"

In [65]:
cifar10.maybe_download_and_extract()  ## Extracting the CIFAR-10 datset

Data has apparently already been downloaded and unpacked.


In [66]:
class_names = cifar10.load_class_names()  ## Getting to know about the 10 classes that is there in the CIFAR-10 dataset.

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


In [80]:
images_train, cls_train, labels_train = cifar10.load_training_data()  ## Lodaing the training dataset
images_test, cls_test, labels_test = cifar10.load_test_data()  ## Loading the testing dataset

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [ ]:
## Here we are defining the architecture of our CNN for CIFAR-10 dataset.
## n_cov1 , n_cov2 --> convulational layers.
## cov1_k , cov2_k --> filters of convulational layers 1 and 2 respectively.
## n_hidden--> Dense Layer
## n_out--> Output Layer

In [81]:
input_width=32     
input_height=32
input_channels=3

input_pixels=32*32*3
n_conv1=32
n_conv2=64
conv1_k=5
conv2_k=5

max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

stride_conv1=1
stride_conv2=1

input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [ ]:
## Now we are reshaping our training and testing data into from 3D to 2D.

In [82]:
images_train=np.reshape(images_train,(images_train.shape[0],input_pixels))
images_test=np.reshape(images_test,(images_test.shape[0],input_pixels))

In [ ]:
## Here we are initialising the different weights and biases of different layers by the random values.

In [83]:
weights={
    
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo"  : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases={
    
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo"  : tf.Variable(tf.random_normal([n_out]))
}

In [ ]:
## Now here data is being acted by the convulational layers.

In [84]:
def conv(x,weights,bias,stride=1):
    out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,stride,stride,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out

In [ ]:
## Pooling Layer is added after the convulational layer which basically tries to reduce the size of the images.

In [85]:
def max_pooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [ ]:
## In this function we are doing forward propagation .
## At first data is passed through the two convulational layers and pooling layers respectively .
## Then we are reshaping our data before passing through the dense layer.
## After data is being passed through the dense layer we are getting the output which is a vector of size 10.

In [93]:
def cnn(x,weights,biases):
    
    x=tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
    conv1=conv(x,weights["wc1"],biases["bc1"],stride_conv1)
    conv1_pool=max_pooling(conv1,max_pool1_k)
    conv2=conv(conv1_pool,weights["wc2"],biases["bc2"],stride_conv2)
    conv2_pool=max_pooling(conv2,max_pool2_k)
    
    hidden_input=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
    hidden_output_before_activation=tf.add(tf.matmul(hidden_input,weights["wh1"]),biases["bh1"])
    hidden_output=tf.nn.relu(hidden_output_before_activation)
    output=tf.add(tf.matmul(hidden_output,weights["wo"]),biases["bo"])
    
    return output

In [ ]:
## Here in this cell we are sending the data for training and prediction.

In [94]:
x=tf.placeholder("float",[None,input_pixels])
y=tf.placeholder(tf.int32,[None,n_out])

pred=cnn(x,weights,biases)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
## Now we are backpropogating our code to find the best values of weights and biases which gives minimum cost . T
##  We are sending our data in batches in every iterations.

In [95]:
batch_size=100

for i in range(25):
    num_batches=int(labels_train.shape[0]/batch_size)
    for j in range(num_batches):
        c,_=sess.run([cost,optimize],feed_dict={x:images_train,y:labels_train})

In [ ]:
## Now Finally! we got our predictions and we now are comparing with the testing output and checking the accuracy of
## our predictions.

In [97]:
predictions = tf.argmax(pred, 1)
correct_labels=[]
for i in range(labels_test.shape[0]):
    for j in range(labels_test.shape[1]):
        if(labels_test[i][j]==1):
            correct_labels.append(j)

correct_labels=np.array(correct_labels)
correct_labels=np.reshape(correct_labels,(labels_test.shape[0]))

correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds = sess.run([predictions,correct_predictions], feed_dict={x:images_test,y:labels_test})
correct_preds.sum()